In [1]:
import pandas as pd
import os
import re
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
import shutil
from scipy import stats
from scipy.stats import norm, skew, f, ttest_ind, ttest_rel, mannwhitneyu, levene, bartlett
import requests
import random
import researchpy as rp
import numpy as np
from sklearn.svm import OneClassSVM
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
cleanDataFolderPath = "/Users/qilu/Desktop/Advanticsys/Data/Output_Data/"
predictDataPath = "/Users/qilu/Desktop/Advanticsys/Data/Output_Data/"

In [4]:
# find and read combined clean data and prediction data and create dataframe
actualCleanDatafile = 'combined_clean_data_by_minute.csv'
predictDatafile = 'energy_prediction.csv'

df_actual = pd.read_csv(cleanDataFolderPath + actualCleanDatafile,index_col=None, header=0)
df_prediction = pd.read_csv(predictDataPath + predictDatafile,index_col=None, header=0)

In [5]:
# df_actual.tail()

In [6]:
# df_prediction.tail()

In [7]:
df = pd.merge(df_actual, df_prediction, how='left', on=["Timestamp", "Inverter"])

In [8]:
# Covert date column into datetime format
df['Date'] = pd.to_datetime(df['Date'])

In [9]:
# get the date part
df['Only_Date'] = df['Date'].dt.date

In [10]:
# get the hour part
df['Hour'] = df['Date'].dt.hour

In [11]:
# get the minute part
df['Minute'] = df['Date'].dt.minute

In [12]:
# df.head()

In [13]:
df.tail()

,Date,Timestamp,Inverter,Energy,Total_Energy,Inv_Temp,Wms_Temp,Wms_Irr,Prediction,Only_Date,Hour,Minute
3100141,2020-12-31 16:23:00,1609411980000,Inv10,1.0,48770.0,36.7,24.8,172.0,NaN,2020-12-31,16,23
3100142,2020-12-31 16:24:00,1609412040000,Inv10,1.0,48770.0,36.7,24.9,170.0,NaN,2020-12-31,16,24
3100143,2020-12-31 16:25:00,1609412100000,Inv10,2.0,48770.0,36.7,24.8,168.0,NaN,2020-12-31,16,25
3100144,2020-12-31 16:28:00,1609412280000,Inv10,1.0,48771.0,36.7,24.6,163.0,NaN,2020-12-31,16,28
3100145,2020-12-31 16:29:00,1609412340000,Inv10,1.0,48771.0,36.7,24.6,160.0,NaN,2020-12-31,16,29


In [14]:
# set formula to allocate minutes into 15 mins interval
def quarter(x):
    if x <= 15:
        return 15
    elif x <= 30:
        return 30
    elif x <= 45:
        return 45
    else:
        return 59

In [15]:
df.loc[:, 'Quarter'] = df.Minute.apply(lambda x: quarter(x))

In [16]:
df.head()

,Date,Timestamp,Inverter,Energy,Total_Energy,Inv_Temp,Wms_Temp,Wms_Irr,Prediction,Only_Date,Hour,Minute,Quarter
0,2021-01-01 07:25:00,1609466100000,Inv01,1.0,69999.0,35.1,17.5,107.0,NaN,2021-01-01,7,25,30
1,2021-01-01 07:26:00,1609466160000,Inv01,1.0,69999.0,35.2,17.6,109.0,NaN,2021-01-01,7,26,30
2,2021-01-01 07:27:00,1609466220000,Inv01,1.0,69999.0,35.3,17.7,112.0,NaN,2021-01-01,7,27,30
3,2021-01-01 07:28:00,1609466280000,Inv01,1.0,70000.0,35.4,17.8,113.0,NaN,2021-01-01,7,28,30
4,2021-01-01 07:29:00,1609466340000,Inv01,1.0,70000.0,35.6,17.8,114.0,NaN,2021-01-01,7,29,30


In [17]:
# Combine minutes into 15 min interval
df = df.drop(['Date', 'Timestamp', 'Minute'], axis=1)
df.rename(columns={"Quarter": "Minute"}, inplace=True)

In [18]:
df.head()

,Inverter,Energy,Total_Energy,Inv_Temp,Wms_Temp,Wms_Irr,Prediction,Only_Date,Hour,Minute
0,Inv01,1.0,69999.0,35.1,17.5,107.0,NaN,2021-01-01,7,30
1,Inv01,1.0,69999.0,35.2,17.6,109.0,NaN,2021-01-01,7,30
2,Inv01,1.0,69999.0,35.3,17.7,112.0,NaN,2021-01-01,7,30
3,Inv01,1.0,70000.0,35.4,17.8,113.0,NaN,2021-01-01,7,30
4,Inv01,1.0,70000.0,35.6,17.8,114.0,NaN,2021-01-01,7,30


In [19]:
# create a column combine time and inverter No.
df['Inv_Date_Hour_Min'] = df['Inverter'].astype(str) + '_' + df['Only_Date'].astype(str) + '_' + df['Hour'].astype(str) + '_' + df['Minute'].astype(str)

In [20]:
# group data into 15 mins interval
agg_functions = {'Inv_Date_Hour_Min': 'first',
                 'Inverter': 'first',
                 'Energy': 'sum',
                 'Total_Energy': 'max',     
                 'Inv_Temp': 'mean',                 
                 'Wms_Temp': 'mean',
                 'Wms_Irr': 'max',                 
                 'Prediction': 'sum',                   
                 'Only_Date': 'first',
                 'Hour': 'first',                                     
                 'Minute': 'first',}
#create new DataFrame by combining rows with same id values
df = df.groupby(df['Inv_Date_Hour_Min']).aggregate(agg_functions)   
df = df.drop(['Inv_Date_Hour_Min'], axis=1)   


In [21]:
# create date column for report building
df['Date'] = df['Only_Date'].astype(str) +" "+ df['Hour'].astype(str) + ':' + df['Minute'].astype(str) + ':' + '00'

In [22]:
df = df.drop(['Hour', 'Minute'], axis=1)

In [23]:
df.head()

,Inverter,Energy,Total_Energy,Inv_Temp,Wms_Temp,Wms_Irr,Prediction,Only_Date,Date
Inv_Date_Hour_Min,,,,,,,,,
Inv01_2020-03-15_12_45,Inv01,1.0,12332.0,34.6,36.7,944.0,0.0,2020-03-15,2020-03-15 12:45:00
Inv01_2020-03-15_12_59,Inv01,1.0,12333.0,41.6,35.4,949.0,0.0,2020-03-15,2020-03-15 12:59:00
Inv01_2020-03-15_13_15,Inv01,1.0,12334.0,44.9,34.5,949.0,0.0,2020-03-15,2020-03-15 13:15:00
Inv01_2020-03-15_13_30,Inv01,1.0,12335.0,47.7,34.7,930.0,0.0,2020-03-15,2020-03-15 13:30:00
Inv01_2020-03-15_13_45,Inv01,1.0,12335.0,48.8,35.4,905.0,0.0,2020-03-15,2020-03-15 13:45:00


In [24]:
# create comparing period label
df['Period_Label'] = ''

In [25]:
# setup periods to compare the past 10 days v the previous 10 days v same 10 days in the previous month v same 10 days in the previous year
current_end_date = df['Only_Date'].max()
current_start_date = current_end_date - pd.DateOffset(days=10)
previous_start_date = current_end_date - pd.DateOffset(days=20)
previous_month_end_date = current_end_date - pd.DateOffset(months=1)
previous_month_start_date = previous_month_end_date - pd.DateOffset(days=10)
previous_year_end_date = current_end_date - pd.DateOffset(years=1)
previous_year_start_date = previous_year_end_date - pd.DateOffset(days=10)

In [26]:
df.loc[(df['Only_Date']<=current_end_date) & (df['Only_Date']>current_start_date), 'Period_Label'] = 'current_period'

In [27]:
df.loc[(df['Only_Date']<=current_start_date) & (df['Only_Date']>previous_start_date), 'Period_Label'] = 'previous_period'

In [28]:
df.loc[(df['Only_Date']<=previous_month_end_date) & (df['Only_Date']>previous_month_start_date), 'Period_Label'] = 'previous_month'

In [29]:
df.loc[(df['Only_Date']<=previous_year_end_date) & (df['Only_Date']>previous_year_start_date), 'Period_Label'] = 'previous_year'

In [30]:
df = df.drop(['Only_Date'], axis=1)

In [31]:
df.tail()

,Inverter,Energy,Total_Energy,Inv_Temp,Wms_Temp,Wms_Irr,Prediction,Date,Period_Label
Inv_Date_Hour_Min,,,,,,,,,
Inv10_2021-12-31_8_59,Inv10,8.0,107245.0,32.287500,20.737500,176.0,0.0,2021-12-31 8:59:00,current_period
Inv10_2021-12-31_9_15,Inv10,13.0,107247.0,32.825000,21.241667,243.0,0.0,2021-12-31 9:15:00,current_period
Inv10_2021-12-31_9_30,Inv10,29.0,107250.0,33.453333,22.346667,662.0,0.0,2021-12-31 9:30:00,current_period
Inv10_2021-12-31_9_45,Inv10,34.0,107253.0,34.620000,23.386667,680.0,0.0,2021-12-31 9:45:00,current_period
Inv10_2021-12-31_9_59,Inv10,26.0,107256.0,35.621429,23.142857,405.0,0.0,2021-12-31 9:59:00,current_period


In [32]:
# conduct statistical analysis, compare variance and mean between the past 10 days v previous periods
# perform hypotheses testing on variance and mean to check if energy production is significant less than previous periods with higher irradiance level
def statisticalAnalysis(df, invNo):
       
    df = df[(df[['Inverter']] == invNo).all(axis=1)] 

    df_c = df[(df[['Period_Label']] == 'current_period').all(axis=1)]   
    df_p = df[(df[['Period_Label']] == 'previous_period').all(axis=1)]   
    df_pm = df[(df[['Period_Label']] == 'previous_month').all(axis=1)]   
    df_py = df[(df[['Period_Label']] == 'previous_year').all(axis=1)]    
    
    # c: current, ce: current energy, ci: current irradiance, cit: current inv temp, cwt: current wms temp
    # p: previous, pe: previous energy, pi: previous irradiance, pit: previous inv temp, pwt: previous wms temp
    # pm: previous month, pme: previous month energy, pmi: previous month irradiance, pmit: previous month inv temp, pmwt: previous month wms temp
    # py: previous year, pye: previous year energy, pyi: previous year irradiance, pyit: previous year inv temp, pywt: previous year wms temp
    
    ce = df_c['Energy']
    ci = df_c['Wms_Irr']

    pe = df_p['Energy']
    pi = df_p['Wms_Irr']

    pme = df_pm['Energy']
    pmi = df_pm['Wms_Irr']

    pye = df_py['Energy']
    pyi = df_py['Wms_Irr']
    
    #normality test - <0.05 not normal distribution, >=0.05 normal
    shapiro_ce = stats.shapiro(ce).pvalue
    shapiro_ci = stats.shapiro(ci).pvalue
    
    shapiro_pe = stats.shapiro(pe).pvalue
    shapiro_pi = stats.shapiro(pi).pvalue
    
    shapiro_pme = stats.shapiro(pme).pvalue
    shapiro_pmi = stats.shapiro(pmi).pvalue
    
    shapiro_pye = stats.shapiro(pye).pvalue
    shapiro_pyi = stats.shapiro(pyi).pvalue    
     
    
    # equal variance (levene) test, <0.05 not equal, >=0.05 equal
    var_cpe = stats.levene(ce, pe).pvalue
    var_cpme = stats.levene(ce, pme).pvalue
    var_cpye = stats.levene(ce, pye).pvalue 
    
    var_cpi = stats.levene(ci, pi).pvalue
    var_cpmi = stats.levene(ci, pmi).pvalue
    var_cpyi = stats.levene(ci, pyi).pvalue        
    
    # mean comparison (mannwhitneyu) test, <0.05 current less, >=0.05 current not less
    mean_cpe = stats.mannwhitneyu(ce, pe, alternative="less").pvalue
    mean_cpme = stats.mannwhitneyu(ce, pme, alternative="less").pvalue
    mean_cpye = stats.mannwhitneyu(ce, pye, alternative="less").pvalue

    mean_cpi = stats.mannwhitneyu(ci, pi, alternative="less").pvalue
    mean_cpmi = stats.mannwhitneyu(ci, pmi, alternative="less").pvalue
    mean_cpyi = stats.mannwhitneyu(ci, pyi, alternative="less").pvalue


    cvp = ''
    cvpe = ''
    if mean_cpe < 0.05 and var_cpe >= 0.05:
        cvpe = 'less' 
    elif mean_cpe < 0.05 and var_cpe < 0.05:
        cvpe = 'less_with_variation'
    elif mean_cpe >= 0.05:
        cvpe = 'not_less'    
        
    cvpi = ''
    if mean_cpi < 0.05 and var_cpi >= 0.05:
        cvpi = 'less'
    elif mean_cpi < 0.05 and var_cpi < 0.05:
        cvpi = 'less_with_variation'
    elif mean_cpi >= 0.05:
        cvpi = 'not_less'    
        
    cvp = cvpe + ' ' + cvpi      
        
    cvpm = ''
    cvpme = ''
    if mean_cpme < 0.05 and var_cpme >= 0.05:
        cvpme = 'less'
    elif mean_cpme < 0.05 and var_cpme < 0.05:
        cvpme = 'less_with_variation'
    elif mean_cpme >= 0.05:
        cvpme = 'not_less'    
        
    cvpmi = ''
    if mean_cpmi < 0.05 and var_cpmi >= 0.05:
        cvpmi = 'less'
    elif mean_cpmi < 0.05 and var_cpmi < 0.05:
        cvpmi = 'less_with_variation'
    elif mean_cpmi >= 0.05:
        cvpmi = 'not_less'    
        
    cvpm = cvpme + ' ' + cvpmi      
        
    cvpy = ''
    cvpye = ''
    if mean_cpye < 0.05 and var_cpye >= 0.05:
        cvpye = 'less'
    elif mean_cpye < 0.05 and var_cpye < 0.05:
        cvpye = 'less_with_variation'
    elif mean_cpye >= 0.05:
        cvpye = 'not_less'    
        
    cvpyi = ''
    if mean_cpyi < 0.05 and var_cpyi >= 0.05:
        cvpyi = 'less'
    elif mean_cpyi >= 0.05 and var_cpyi < 0.05:
        cvpyi = 'less_with_variation'
    elif mean_cpyi >= 0.05:
        cvpyi = 'not_less'    
        
    cvpy = cvpye + ' ' + cvpyi  
    
    
    
    df['Current_vs_Previous'] = ''
    df['Current_vs_Previous_Month'] = ''
    df['Current_vs_Previous_Year'] = ''
    df.loc[df['Period_Label'] == 'current_period', 'Current_vs_Previous'] = cvp
    df.loc[df['Period_Label'] == 'current_period', 'Current_vs_Previous_Month'] = cvpm
    df.loc[df['Period_Label'] == 'current_period', 'Current_vs_Previous_Year'] = cvpy
  

    return df
    
    



In [33]:
# run for all inverters and combine all data into 15 mins interval
df_15min_combine = pd.DataFrame()
invNo = ['Inv01',
         'Inv02',
         'Inv03',
         'Inv04',
         'Inv05',
         'Inv06',
         'Inv07',
         'Inv08',
         'Inv09',
         'Inv10']

for inv in invNo:
    df_15min_combine = df_15min_combine.append(statisticalAnalysis(df,inv), ignore_index = True)


In [34]:
df_15min_combine.tail()

,Inverter,Energy,Total_Energy,Inv_Temp,Wms_Temp,Wms_Irr,Prediction,Date,Period_Label,Current_vs_Previous,Current_vs_Previous_Month,Current_vs_Previous_Year
241932,Inv10,8.0,107245.0,32.287500,20.737500,176.0,0.0,2021-12-31 8:59:00,current_period,not_less not_less,not_less not_less,less_with_variation not_less
241933,Inv10,13.0,107247.0,32.825000,21.241667,243.0,0.0,2021-12-31 9:15:00,current_period,not_less not_less,not_less not_less,less_with_variation not_less
241934,Inv10,29.0,107250.0,33.453333,22.346667,662.0,0.0,2021-12-31 9:30:00,current_period,not_less not_less,not_less not_less,less_with_variation not_less
241935,Inv10,34.0,107253.0,34.620000,23.386667,680.0,0.0,2021-12-31 9:45:00,current_period,not_less not_less,not_less not_less,less_with_variation not_less
241936,Inv10,26.0,107256.0,35.621429,23.142857,405.0,0.0,2021-12-31 9:59:00,current_period,not_less not_less,not_less not_less,less_with_variation not_less


In [35]:
# save the combined file use for final Tableau dashboard
df_15min_combine.to_csv('/Users/qilu/Desktop/Advanticsys/Data/Output_Data/15min_combine_data_for_dashboard.csv', index=False)